In [13]:
import itertools
import re

def solve(formula):
    """Given a formula like 'NUM + BER = PLAY', fill in digits to solve it.
    Generate all valid digit-filled-in strings."""
    return filter(valid, letter_replacements(formula))

def letter_replacements(formula):
    """All possible replacements of letters with digits in formula."""
    formula = formula.replace(' = ', ' == ') # Allow = or ==
    letters = cat(set(re.findall('[A-Z]', formula)))
    for digits in itertools.permutations('1234567890', len(letters)):
        yield formula.translate(str.maketrans(letters, cat(digits)))

def valid(exp):
    """Expression is valid iff it has no leading zero, and evaluates to true."""
    try:
        return not leading_zero(exp) and eval(exp) is True
    except ArithmeticError:
        return False
    
cat = ''.join # Function to concatenate strings
    
leading_zero = re.compile(r'\b0[0-9]').search # Function to check for illegal number

In [16]:
next(solve('NUM + BER = PLAY'))

'675 + 423 == 1098'

In [15]:
%time len(set(solve('NUM + BER = PLAY')))

CPU times: user 27.5 s, sys: 41.7 ms, total: 27.5 s
Wall time: 27.8 s


96

In [17]:
%prun next(solve('NUM + BER = PLAY'))

In [18]:
def compile_formula(formula, verbose=False):
    """Compile formula into a function.   Also return letters found, as a str,
    in same order as parms of function. For example, 'YOU == ME**2' returns
    (lambda E,M,O,U,Y: M and Y and ((100*Y+10*O+U) == (10*M+E)**2), 'YMEUO'"""
    formula      = formula.replace(' = ', ' == ')
    letters      = cat(sorted(set(re.findall('[A-Z]', formula))))
    firstletters = sorted(set(re.findall(r'\b([A-Z])[A-Z]', formula)))
    body         = re.sub('[A-Z]+', compile_word, formula)
    body         = ' and '.join(firstletters + [body])
    fn = 'lambda {}: {}'.format(','.join(letters), body)
    if verbose: print(fn)
    assert len(letters) <= 10
    return eval(fn), letters

def compile_word(matchobj):
    "Compile the word 'YOU' as '(100*Y + 10*O + U)'."
    word = matchobj.group()
    terms = reversed([mult(10**i, L) for (i, L) in enumerate(reversed(word))])
    return '(' + '+'.join(terms) + ')'

def mult(factor, var): return var if factor == 1 else str(factor) + '*' + var

In [19]:
compile_formula("YOU == ME**2", verbose=True)

lambda E,M,O,U,Y: M and Y and (100*Y+10*O+U) == (10*M+E)**2


(<function __main__.<lambda>>, 'EMOUY')

In [20]:
compile_formula("NUM + BER = PLAY", verbose=True)

lambda A,B,E,L,M,N,P,R,U,Y: B and N and P and (100*N+10*U+M) + (100*B+10*E+R) == (1000*P+100*L+10*A+Y)


(<function __main__.<lambda>>, 'ABELMNPRUY')

In [21]:
def faster_solve(formula):
    """Given a formula like 'NUM + BER == PLAY', fill in digits to solve it.
    This version precompiles the formula and generates all digit-filled-in strings."""
    fn, letters = compile_formula(formula)
    for digits in itertools.permutations((1,2,3,4,5,6,7,8,9,0), len(letters)):
        try:
            if fn(*digits):
                yield formula.translate(str.maketrans(letters, cat(map(str, digits))))
        except ArithmeticError: 
            pass

In [22]:
next(faster_solve('NUM + BER = PLAY'))

'587 + 439 = 1026'

In [23]:
%time len(list(faster_solve('NUM + BER = PLAY')))

CPU times: user 1.95 s, sys: 5.07 ms, total: 1.96 s
Wall time: 1.97 s


96

In [25]:
examples = """
NUM + BER = PLAY
YOU = ME ** 2
SEND + MORE = MONEY
PI * R**2 = AREA
WRONG + WRONG = RIGHT
WRIGHT + WRIGHT = TO * FLY + FLIGHT
TWO + TWENTY = TWELVE + TEN
A**2 + B**2 = C**2
AYH**2 + BEE**2 = SEE**2
RAMN = R**3 + RM**3 = N**3 + RX**3
MON-EY = EVIL**(1/2)
ALPHABET + LETTERS = SCRABBLE
SIN**2 + COS**2 = UNITY
POTATO + TOMATO = PUMPKIN
ODD * ODD = FREAKY
DOUBLE + DOUBLE + TOIL = TROUBLE
WASH + YOUR = HANDS
SPEED + LIMIT = FIFTY
TERRIBLE + NUMBER = THIRTEEN
SEVEN + SEVEN + SIX = TWENTY
EIGHT + EIGHT + TWO + ONE + ONE = TWENTY
ELEVEN + NINE + FIVE + FIVE = THIRTY
NINE + SEVEN + SEVEN + SEVEN = THIRTY
FOURTEEN + TEN + TEN + SEVEN = FORTYONE
HAWAII + IDAHO + IOWA + OHIO = STATES
VIOLIN * 2 + VIOLA = TRIO + SONATA
SEND + A + TAD + MORE = MONEY
ZEROES + ONES = BINARY
DCLIZ + DLXVI = MCCXXV
COUPLE + COUPLE = QUARTET
FISH + N + CHIPS = SUPPER
SATURN + URANUS + NEPTUNE + PLUTO = PLANETS
PLUTO not in {PLANETS}
EARTH + AIR + FIRE + WATER = NATURE
TWO * TWO = SQUARE
HIP * HIP = HURRAY
NORTH / SOUTH = EAST / WEST
NAUGHT ** 2 = ZERO ** 3
I + THINK + IT + BE + THINE = INDEED
DO + YOU + FEEL = LUCKY
WELL - DO + YOU = PUNK
NOW + WE + KNOW + THE = TRUTH
SORRY + TO + BE + A + PARTY = POOPER
SORRY + TO + BUST + YOUR = BUBBLE
""".strip().splitlines()

def run(examples):
    for e in examples:
        print('{:45}| {}'.format(e, next(faster_solve(e))))
    return len(examples)
        
%time run(examples)

NUM + BER = PLAY                             | 587 + 439 = 1026
YOU = ME ** 2                                | 576 = 24 ** 2
SEND + MORE = MONEY                          | 9567 + 1085 = 10652
PI * R**2 = AREA                             | 96 * 7**2 = 4704
WRONG + WRONG = RIGHT                        | 37081 + 37081 = 74162
WRIGHT + WRIGHT = TO * FLY + FLIGHT          | 413058 + 413058 = 82 * 769 + 763058
TWO + TWENTY = TWELVE + TEN                  | 784 + 781279 = 781351 + 712
A**2 + B**2 = C**2                           | 3**2 + 4**2 = 5**2
AYH**2 + BEE**2 = SEE**2                     | 760**2 + 522**2 = 922**2
RAMN = R**3 + RM**3 = N**3 + RX**3           | 1729 = 1**3 + 12**3 = 9**3 + 10**3
MON-EY = EVIL**(1/2)                         | 108-42 = 4356**(1/2)
ALPHABET + LETTERS = SCRABBLE                | 17531908 + 7088062 = 24619970
SIN**2 + COS**2 = UNITY                      | 235**2 + 142**2 = 75389
POTATO + TOMATO = PUMPKIN                    | 168486 + 863486 = 1031972
ODD * OD

44